In [1]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 7.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 5.0 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=4993432149f1a39cca261570ce29e1bbc7f475419acccd2ede06476ff3b412d1
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [2]:
import os
import joblib
import numpy as np
import pandas as pd
import polars as pl

import pandas.api.types
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import lightgbm as lgb

import lifelines
from lifelines.utils import concordance_index

import sys

print(sys.version)
for i in [np, pd, pl, mpl, sns, lifelines, sklearn]:
    try:
        print(i.__name__, i.__version__)
    except:
        print(i.__name__)

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
numpy 1.26.4
pandas 2.2.3
polars 1.9.0
matplotlib 3.7.5
seaborn 0.12.2
lifelines 0.30.0
sklearn 1.2.2


In [3]:
import dproc, sgutil, sgpp, sgml, custpp
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer

In [4]:
X_4 = [
    'psych_disturb', 'diabetes', 'arrhythmia', 'renal_issue', 'pulm_severe', 'obesity', 'hepatic_severe', 
    'prior_tumor', 'peptic_ulcer','rheum_issue', 'hepatic_mild', 'cardiac','pulm_moderate'
]

X_int_fm = [
    'comorbidity_score', 'hla_high_res_10', 'hla_high_res_6', 'hla_high_res_8', 'hla_low_res_10',
    'hla_low_res_6', 'hla_low_res_8', 'hla_match_a_high', 'hla_match_a_low', 'hla_match_b_high',
    'hla_match_b_low', 'hla_match_drb1_low', 'hla_match_c_high', 'hla_match_c_low', 'hla_match_dqb1_high', 'hla_match_dqb1_low',
    'hla_match_drb1_high', 'hla_nmdp_6', 'karnofsky_score'
]

X_2 = [
    ('graft_type', {'peripheral blood': 0, 'bone marrow': 1}),
    ('prod_type', {'pb': 0, 'bm': 1}),
    ('vent_hist', {'no': -1, 'yes': 1}),
    ('rituximab', {'no': -1, 'yes': 1}),
    ('mrd_hct', {'negative': -1, 'positive': 1}),
    ('in_vivo_tcd', {'no': -1, 'yes': 1}),
    ('melphalan_dose', {'n/a, mel not given': -1, 'mel': 1})
]
X_na = X_4 +  X_int_fm +  ['donor_age']
X_nom_na = ['cmv_status', 'conditioning_intensity', 'cyto_score', 'cyto_score_detail', 'donor_related',
    'dri_score', 'ethnicity', 'gvhd_proph', 'sex_match', 'tce_div_match', 'tce_imm_match', 'tce_match'
]
X_nom_nna = ['prim_disease_hct', 'race_group', 'tbi_status']
p1_test = make_pipeline(
    sgpp.PolarsProcessor({'ID': pl.Int64}),
    sgpp.PandasCoverter(index_col = 'ID'),
    custpp.CIBMTTransformer(X_2, X_4, X_na, X_nom_na + X_nom_nna),
    sgpp.ApplyWrapper(SimpleImputer(strategy='most_frequent').set_output(transform='pandas'), X_int_fm, postfix = '_fm'),
    sgpp.ApplyWrapper(SimpleImputer(strategy='mean').set_output(transform='pandas'), ['donor_age'], postfix = '_fm'),
    sgpp.ApplyWrapper(sgpp.CatArrangerFreq(1, 'na', 'na'), X_nom_na),
    sgpp.ApplyWrapper(sgpp.CatOOVFilter(), X_nom_nna),
)
df_train = p1_test.fit_transform(['data/train.csv'])
joblib.dump(p1_test, os.path.join('model', 'p1_test.joblib'))

['model/p1_test.joblib']

In [8]:
"""p1_test = joblib.load(os.path.join('model', 'p1_test.joblib'))
df_train_2 = pd.read_csv('data/train_2.csv', index_col = 'ID', dtype = {
    **{i: 'category' for i, _ in X_2},
    **{i: 'category' for i in X_4},
    **{i: 'category' for i in X_nom_na},
    **{i: 'category' for i in X_nom_nna}}
)"
df_train_2 = p1_test.transform(['data/train_2.csv'])
df_train_2.head()"""

'p1_test = joblib.load(os.path.join(\'model\', \'p1_test.joblib\'))\ndf_train_2 = pd.read_csv(\'data/train_2.csv\', index_col = \'ID\', dtype = {\n    **{i: \'category\' for i, _ in X_2},\n    **{i: \'category\' for i in X_4},\n    **{i: \'category\' for i in X_nom_na},\n    **{i: \'category\' for i in X_nom_nna}}\n)"\ndf_train_2 = p1_test.transform([\'data/train_2.csv\'])\ndf_train_2.head()'

In [9]:
X_bool = ['graft_type', 'prod_type']
X_tri = [
    'arrhythmia', 'cardiac', 'diabetes', 'hepatic_mild', 'hepatic_severe',
    'in_vivo_tcd', 'melphalan_dose', 'mrd_hct', 'obesity', 'peptic_ulcer',
    'prior_tumor', 'psych_disturb', 'pulm_moderate', 'pulm_severe', 'renal_issue',
    'rheum_issue', 'rituximab', 'vent_hist'
]
X_nom = [
    'cmv_status', 'conditioning_intensity', 'cyto_score', 'cyto_score_detail', 'donor_related',
    'dri_score', 'ethnicity', 'gvhd_proph', 'prim_disease_hct', 'race_group', 'sex_match',
    'tbi_status', 'tce_div_match', 'tce_imm_match', 'tce_match'
]
X_na = [
    'arrhythmia_na', 'cardiac_na', 'diabetes_na', 'hepatic_mild_na', 'hepatic_severe_na',
    'obesity_na', 'peptic_ulcer_na', 'prior_tumor_na', 'psych_disturb_na', 'pulm_moderate_na',
    'pulm_severe_na', 'renal_issue_na', 'rheum_issue_na'
]
X_cont = ['age_at_hct', 'donor_age']
X_int = [
    'comorbidity_score', 'hla_high_res_10', 'hla_high_res_6', 'hla_high_res_8', 'hla_low_res_10',
    'hla_low_res_6', 'hla_low_res_8', 'hla_match_a_high', 'hla_match_a_low', 'hla_match_b_high',
    'hla_match_b_low', 'hla_match_drb1_low', 'hla_match_c_high', 'hla_match_c_low', 'hla_match_dqb1_high', 'hla_match_dqb1_low',
    'hla_match_drb1_high', 'hla_nmdp_6', 'karnofsky_score', 'year_hct'
]


In [10]:
from lifelines import KaplanMeierFitter
def transform_survival_probability(df, time_col='efs_time', event_col='efs'):
    kmf = KaplanMeierFitter()
    kmf.fit(df[time_col], df[event_col])
    y = kmf.survival_function_at_times(df[time_col]).values
    return y
df_train['kmf'] = transform_survival_probability(df_train, time_col='efs_time', event_col='efs')

In [11]:
def score(df, prds):
    return df.groupby('race_group', observed=True).apply(
        lambda x: concordance_index(x['efs_time'], -prds.loc[x.index], x['efs']), include_groups=False
    ).pipe(
        lambda x: float(x.mean() - x.std(ddof=0))
    )

In [12]:
def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict(df[X]), index = df.index),
    'score_func': lambda df, prds: score,
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'y': 'kmf',
}

lgb_adapter = sgml.LGBMAdapter(lgb.LGBMRegressor)

In [13]:
hparams = {
    'model_params': {'num_leaves':  64, 'n_estimators':  300},
    'X_num': X_tri + X_cont + X_int + X_na + X_bool, 'X_cat': X_nom
}
result = sgml.train(df_train, hparams, config, lgb_adapter)

Round:   0%|          | 0/300 [00:00<?, ?it/s]

In [14]:
score(
    df_train,
    pd.Series(
        make_pipeline(result[0]['preprocessor'], result[0]['model']).predict(df_train[result[1]]), index = df_train.index
    )
)

0.8373568213852252

In [15]:
joblib.dump(result, os.path.join('model', 'lgb_test.joblib'))

['model/lgb_test.joblib']